In [1]:
# -*- coding:utf-8 -*-
import sys, os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

/Users/sunchao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/sunchao/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
os.chdir("/Users/sunchao/Desktop/Guo/")

In [3]:
train_data = pd.read_csv("./X_train.csv", header=0)
feature_columns = ["MYCT","MMIN","MMAX","CACH","CHMIN","CHMAX"]
# remove id columns
train_data = train_data.iloc[:,1:]
# remove null columns
train_data.drop(['Unnamed: 7'],axis=1, inplace=True)

In [4]:
# split the train data of train-data and validation-data
X_train,X_val,y_train,y_val = train_test_split(train_data[feature_columns],train_data.PRP,train_size=0.9)

X_train = np.asarray(X_train)
X_train = scale(X_train)

X_val = np.asarray(X_val)
X_val = scale(X_val)

y_train = np.asarray(y_train).reshape((-1,1))
y_train = scale(y_train)

y_val = np.asarray(y_val).reshape((-1,1))
# y_val = scale(y_val)

/Users/sunchao/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/Users/sunchao/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [5]:
print("自变量---源数据:",train_data[feature_columns].shape, "；  训练集:",X_train.shape, "；  验证集:",X_val.shape)
print("因变量---源数据:",train_data.PRP.shape, "；  训练集:",y_train.shape, "；  验证集:",y_val.shape)

自变量---源数据: (168, 6) ；  训练集: (151, 6) ；  验证集: (17, 6)
因变量---源数据: (168,) ；  训练集: (151, 1) ；  验证集: (17, 1)


In [6]:
# 网络添加函数
def add_layer(inputs, input_size, output_size, activation_function=None):
    with tf.variable_scope("Weights"):
        Weights = tf.Variable(tf.random_normal(shape=[input_size,output_size]), name="weights")
    with tf.variable_scope("biases"):
        biases = tf.Variable(tf.zeros(shape=[1, output_size]) + 0.1, name="biases")
    with tf.name_scope("Wx_plus_b"):
        Wx_plus_b = tf.matmul(inputs, Weights) + biases
    with tf.name_scope("dropout"):
        Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob=keep_prob_s)
    if activation_function is None:
        return Wx_plus_b
    else:
        return activation_function(Wx_plus_b)

In [7]:
# 定义占位符和网络层数
xs = tf.placeholder(shape = [None, X_train.shape[1]], dtype=tf.float32, name="inputs")
ys = tf.placeholder(shape = [None, 1], dtype=tf.float32, name="y_true")
keep_prob_s = tf.placeholder(dtype=tf.float32)

with tf.name_scope("layer_1"):
    l1 = add_layer(xs, 6, 100, activation_function=tf.nn.relu)

with tf.name_scope("hidder_layer1"):
    h_layer1 = add_layer(l1, 100, 1000, activation_function=tf.nn.relu)
        
with tf.name_scope("hidder_layer2"):
    h_layer2 = add_layer(h_layer1, 1000, 100, activation_function=tf.nn.relu)  

with tf.name_scope("hidder_layer3"):
    h_layer3 = add_layer(h_layer2, 100, 10, activation_function=tf.nn.relu)  

with tf.name_scope("y_pred"):
    pred = add_layer(h_layer3, 10, 1)

# 保存pred的操作结果，做恢复使用
pred = tf.add(pred, 0, name="pred")

with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - pred), reduction_indices=[1]))   # MSE误差
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

In [8]:
def fit(X,y,n, keep_prob):
    init = tf.global_variables_initializer()
    feed_dict = {xs:X, ys:y, keep_prob_s:keep_prob}
    with tf.Session() as sess:

        saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter(logdir="nn_log", graph=sess.graph)  #写tensorbord
        sess.run(init)
        for i in range(n):
            _loss, _ = sess.run([loss,train_op], feed_dict=feed_dict)
            
            if i % 100 == 0:
                print("epoch:%d\tloss:%.5f" % (i, _loss))
                y_pred = sess.run(pred, feed_dict = feed_dict)
                rs = sess.run(merged, feed_dict=feed_dict)
                writer.add_summary(summary=rs, global_step=i)  #写tensorbord
                saver.save(sess=sess, save_path="nn_model/nn.model", global_step=i) # 保存模型
        saver.save(sess=sess, save_path="nn_model/nn.model", global_step=n)  # 保存模型 

In [9]:
fit(X=X_train,y=y_train,n=10000,keep_prob=1)

epoch:0	loss:3087697.25000
epoch:100	loss:183.64865
epoch:200	loss:41.75545
epoch:300	loss:14.50969
epoch:400	loss:5.87180
epoch:500	loss:2.80568
epoch:600	loss:1.79247
epoch:700	loss:1.35292
epoch:800	loss:1.11992
epoch:900	loss:0.99900
epoch:1000	loss:0.93859
epoch:1100	loss:0.90704
epoch:1200	loss:0.88981
epoch:1300	loss:0.87997
epoch:1400	loss:0.87396
epoch:1500	loss:0.86857
epoch:1600	loss:0.86483
epoch:1700	loss:0.86831
epoch:1800	loss:0.85937
epoch:1900	loss:0.85699
epoch:2000	loss:0.85446
epoch:2100	loss:0.85201
epoch:2200	loss:0.84979
epoch:2300	loss:0.84770
epoch:2400	loss:0.84546
epoch:2500	loss:0.84515
epoch:2600	loss:0.84171
epoch:2700	loss:0.83969
epoch:2800	loss:0.83778
epoch:2900	loss:0.83584
epoch:3000	loss:0.83372
epoch:3100	loss:0.83154
epoch:3200	loss:0.82923
epoch:3300	loss:2.22281
epoch:3400	loss:0.90200
epoch:3500	loss:0.86589
epoch:3600	loss:0.84948
epoch:3700	loss:0.84114
epoch:3800	loss:0.83627
epoch:3900	loss:1.28928
epoch:4000	loss:0.82536
epoch:4100	loss:0.

In [10]:
# use model to predict valid_data
def predict(X,y,keep_prob):

    with tf.Session() as sess:

        # restore saver
        saver = tf.train.import_meta_graph(meta_graph_or_file="./nn_model/nn.model-10000.meta")
        model_file = tf.train.latest_checkpoint(checkpoint_dir="nn_model")
        saver.restore(sess=sess,save_path=model_file)

        # init graph
        graph = tf.get_default_graph()

        # get placeholder from graph
        xs = graph.get_tensor_by_name("inputs:0")
        ys = graph.get_tensor_by_name("y_true:0")
        keep_prob_s = graph.get_tensor_by_name("keep_prob:0")

        # get operation from graph
        pred = graph.get_tensor_by_name("pred:0")

        # run pred
        feed_dict = {xs: X, ys: y, keep_prob_s: keep_prob}
        y_pred = sess.run(pred,feed_dict=feed_dict)

    return y_pred.reshape(-1)

In [11]:
y_pred = predict(X_val, y_val, 1)

ValueError: cannot add op with name layer_1/Weights/weights/Adam as that name is already used